In [1]:
import findspark
findspark.init()
from pyspark import SparkContext as sc

import os

from pyspark.sql import SQLContext, Row

import pyspark

import sys
import string
from csv import reader
from functools import reduce
from pyspark.sql import functions as f
from collections import defaultdict
import datetime

sys.path.insert(0, '../src')
from tableCollections import TableCollections
import os

In [2]:
data_path = ['data']
parking_filepath = os.sep.join(data_path + ['parking.csv'])

sc = sc(appName="Test_all")
sqlContext = SQLContext(sc)

In [5]:
parkingTable = sqlContext.read.format('csv').options(header='true',inferschema='true').load('/user/ecc290/HW1data/parking-violations-header.csv')

In [7]:
openTable = sqlContext.read.format('csv').options(header='true',inferschema='true').load('/user/ecc290/HW1data/open-violations-header.csv')

In [8]:
tc = TableCollections(sqlContext, sc)

In [9]:
tc.register(parkingTable, "parking")

num metadata file exists for table parking
timestamp metadata file exists for table parking
string metadata file exists for table parking


In [11]:
tc.register(openTable, "open")

num metadata file exists for table open
string metadata file exists for table open


In [10]:
import datetime
tc.timeColWithinRange(datetime.datetime(1994,1,1), datetime.datetime(2018,5,1)).show()

+----------+---------+
|   colName|tableName|
+----------+---------+
|issue_date|  parking|
+----------+---------+



In [14]:
tc.numColWithinRange(-1000000, 10000000).show()

+------------------+---------+
|           colName|tableName|
+------------------+---------+
|violation_location|  parking|
|violation_precinct|  parking|
|   issuer_precinct|  parking|
|       issuer_code|  parking|
|    violation_code|  parking|
|      vehicle_year|  parking|
|  reduction_amount|     open|
|   interest_amount|     open|
|    payment_amount|     open|
|    penalty_amount|     open|
|       fine_amount|     open|
|        amount_due|     open|
|          precinct|     open|
+------------------+---------+



In [16]:
colList= ["parking^violation_location","parking^violation_precinct", "open^reduction_amount", "open^amount_due"]
tc.getNumRange(colList).show()

+---------+------------------+----+-----+
|tableName|           colName| min|  max|
+---------+------------------+----+-----+
|  parking|violation_location|-1.0|967.0|
|  parking|violation_precinct| 0.0|967.0|
|     open|  reduction_amount| 0.0|545.0|
|     open|        amount_due| 0.0|594.1|
+---------+------------------+----+-----+



In [19]:
colList= ["parking^issue_date"]
tc.getTimeRange(colList).show()

+---------+----------+-------------------+-------------------+
|tableName|   colName|                min|                max|
+---------+----------+-------------------+-------------------+
|  parking|issue_date|2016-03-01 00:00:00|2016-03-31 00:00:00|
+---------+----------+-------------------+-------------------+



In [22]:
tc.getSimilarNumCols("parking^violation_location", threshold=0.1).show()

+------------------+---------+------------------+
|           colName|tableName|               iou|
+------------------+---------+------------------+
|violation_location|  parking|               1.0|
|violation_precinct|  parking|0.9989669421487604|
|   issuer_precinct|  parking| 0.973816717019134|
|          precinct|     open|0.8316115702479339|
|        amount_due|     open|0.6137396694214876|
|    payment_amount|     open|0.6099586776859505|
|  reduction_amount|     open|0.5630165289256198|
|       fine_amount|     open| 0.506198347107438|
|      vehicle_year|  parking|0.4671497584541063|
|    violation_code|  parking|0.1012396694214876|
+------------------+---------+------------------+



In [25]:
colList= ["parking^plate_id","open^plate"]
tc.returnIntersecWithinCols(colList).show()

+---------+-----+
|col_value|count|
+---------+-----+
|  12384MG|    2|
|   2AL711|    2|
|   XAGM54|    2|
|  24781MG|    2|
|   EVRB25|    2|
|  31590ME|    2|
|  FZA5037|    2|
|  VFL2464|    2|
|  VKD5120|    2|
|   Y69AVV|    2|
|  FRJ7147|    2|
|  FZK8632|    2|
|  26333MH|    2|
|   XCSB43|    2|
|  92429JY|    2|
|  58730MC|    2|
|  ETU2643|    2|
|  EHD5439|    2|
|  2267754|    2|
|   D57CVL|    2|
+---------+-----+
only showing top 20 rows



In [26]:
colList= ["parking^plate_id","open^plate"]
tc.frequentVals(colList, 5).show()

+----------+----+--------+
| col_value| cnt|col_name|
+----------+----+--------+
|BLANKPLATE|1203|plate_id|
|BLANKPLATE| 872|   plate|
|       N/A| 155|plate_id|
|       N/A| 149|   plate|
|    AP501F| 138|plate_id|
+----------+----+--------+



In [28]:
colList= ["parking^plate_id","open^plate"]
tc.returnOutliers(colList, 0.01)

+---------+---+--------+
|col_value|cnt|col_name|
+---------+---+--------+
|  BML4037|  1|plate_id|
| T655130C|  1|plate_id|
| T649453C|  1|plate_id|
|  CHU7059|  1|plate_id|
|   D57CVL|  1|plate_id|
|  EWX5634|  1|plate_id|
|  30437MA|  1|plate_id|
|  DUA2419|  1|plate_id|
|   Z77FJX|  1|plate_id|
|  ERG9119|  1|plate_id|
|  EWD7040|  1|plate_id|
|  FYW3414|  1|plate_id|
|  DRV8081|  1|plate_id|
|  FRJ7147|  1|plate_id|
|  39034MG|  1|plate_id|
|   Z41DYN|  1|plate_id|
|   N77FNS|  1|plate_id|
|   E91EYH|  1|plate_id|
|  CPZ3300|  1|plate_id|
|  EHD5439|  1|plate_id|
+---------+---+--------+
only showing top 20 rows

+---------+---+--------+
|col_value|cnt|col_name|
+---------+---+--------+
|  GWM2129|  1|   plate|
|  GVS9200|  1|   plate|
|  ACL4445|  1|   plate|
|  GGJ4899|  1|   plate|
|  HBH9968|  1|   plate|
|   595ZSY|  1|   plate|
|  BVS5686|  1|   plate|
|  HDU4451|  1|   plate|
|  20213MA|  1|   plate|
|  EBE1052|  1|   plate|
|  GDN2939|  1|   plate|
|  GFF4794|  1|   plate|

In [32]:
colList= ["parking^plate_id","open^plate"]
withList = ['CPZ3300']
withoutList = ['GTR366A']
tc.colsWithAndWithout(colList, withList, withoutList).show()

+--------+----------+
|col_name|table_name|
+--------+----------+
|   plate|      open|
+--------+----------+



In [34]:
tc.getCardinality(colList).show()

+--------+------+
|col_name| count|
+--------+------+
|   plate|212692|
|plate_id|607520|
+--------+------+



In [35]:
tc.getColsOfDatatype(openTable, 'string')

['plate',
 'license_type',
 'county',
 'state',
 'issuing_agency',
 'violation',
 'violation_status',
 'issue_date',
 'violation_time',
 'judgment_entry_date']

In [38]:
#Necessary to use nltk library
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/jp4989/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [41]:
tc.colsNameSimilarity(openTable, category = None, df2=parkingTable)

TypeError: unorderable types: NoneType() > float()

In [47]:
tc.getColsofCategory(parkingTable, ['registration_state'], 'State_short').show()

+--------+--------+
|category|IsSubset|
+--------+--------+
|     NaN|     NaN|
+--------+--------+



In [48]:
tc.countNullValues(parkingTable)

TypeError: Column is not iterable

In [50]:
colList= ["parking^plate_id","open^plate"]
tc.returnUnionWithinCols(colList).show()

+---------+-----+
|col_value|count|
+---------+-----+
|  ACL4445|    2|
|  GGJ4899|    2|
|  HBH9968|    2|
|  GPL7023|    2|
|  BVS5686|    2|
|  HDU4451|    2|
|  HAY6630|    2|
|  20213MA|    2|
|  GDN2939|    2|
|  HDZ8438|    2|
|  55411JS|    2|
|  CFJ8401|    2|
|  44366MD|    2|
|  VFR2928|    2|
|   XCSB43|    2|
|  DSZ6353|    2|
|  GKL8441|    2|
| T649254C|    2|
|  VBU9385|    2|
|  GKN1233|    2|
+---------+-----+
only showing top 20 rows

